In [1]:
import os
os.environ['SPARK_HOME'] = "/home/user/.application-data/spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'
os.environ['PYSPARK_PYTHON'] = 'python'

from pyspark.sql import *

In [2]:
# Creates the department
department1 = Row(id='123456', name='Computer Science')

In [3]:
# Creates the employee table schema and an employee
Employee = Row('first_name', 'last_name', 'email', 'salary')
employee1 = Employee('Michael', 'Jackson', 'michael.jackson@rusbe.com', 1000000)
employee2 = Employee('Manoel', 'Gome', 'manoel.gome@bluepen.com', 1000000)

In [4]:
# create the DepartmentWithEmployees instance from Department and Employees
departmentWithEmployees = Row(department=department1, employees = [employee1, employee2])

In [5]:
# join dataframes (concatenate)
spark = SparkSession.builder.appName('pyspark-intro').getOrCreate()
accounts = spark.read.option('header', True).csv('../data/accounts.csv')
accounts_dup = accounts.union(accounts)
display(accounts_dup.show(5))

24/09/20 15:11:48 WARN Utils: Your hostname, user-PC resolves to a loopback address: 127.0.1.1; using 192.168.0.27 instead (on interface wlp3s0)
24/09/20 15:11:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 15:11:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------------------+---------+-----------+----------+-------+
|    account_number|      aba|        bic|    opened|balance|
+------------------+---------+-----------+----------+-------+
|CBXD44669851029839|102317138|RWWEGBRO0T5|2009-06-04| 285769|
|GMHS11683964344077|082341703|   TIDCGB2I|2014-06-26| 591000|
|GCQT83212037197512|107809939|HGMJGB9AIFY|2006-08-07|1631126|
|TDDI73474064461675|046709967|VQEAGBT2HP7|2001-03-20|1977359|
|HDVG29919274654139|069217423|BQCSGBADD0Z|2014-11-05| 208204|
+------------------+---------+-----------+----------+-------+
only showing top 5 rows



None

In [6]:
# UDF (User Defined Functions)
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType

def squared(x):
    return float(x) * float(x)

spark.udf.register('squared', squared)
squared_udf = udf(squared, LongType())

df_sqrd = accounts.select('account_number', squared_udf('balance').alias('balance squared'))
display(df_sqrd.show(5))

+------------------+---------------+
|    account_number|balance squared|
+------------------+---------------+
|CBXD44669851029839|           NULL|
|GMHS11683964344077|           NULL|
|GCQT83212037197512|           NULL|
|TDDI73474064461675|           NULL|
|HDVG29919274654139|           NULL|
+------------------+---------------+
only showing top 5 rows



None

In [7]:
# pyspark pandas API
import pyspark.pandas as ps

ps_df = ps.range(10)
pd_df = ps_df.to_pandas()
pd_df.head()

24/09/20 15:12:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
/home/user/.application-data/miniconda3/envs/data-eng/lib/python3.12/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/home/user/.application-data/miniconda3/envs/data-eng/lib/python3.12/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, Panda

,id
0,0
1,1
2,2
3,3
4,4
